In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pathlib
import glob
import tqdm
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import LinearNDInterpolator

from mc2.models.preisach import PreisachModel, analyticalPreisachFunction1, analyticalPreisachFunction2, preisachIntegration

In [ ]:
model = PreisachModel(500, 1)

In [ ]:
A = 4
Hc = 0.01
sigma = 0.03
preisach = analyticalPreisachFunction2(A, Hc, sigma, model.gridX, model.gridY)

# Calculate Everett function from preisach function
everett = preisachIntegration(model.width, preisach)

# Scale Everett function to a maximum value of 1
everett = everett / np.max(everett)

# Calculate linear Interpolator for Everett function
points = np.zeros((everett.size, 2), dtype=np.float64)
points[:, 0] = model.gridX.flatten()
points[:, 1] = model.gridY.flatten()
values = everett.flatten()
everettInterp = LinearNDInterpolator(points, values)
model.setEverettFunction(everettInterp)

# show everett function of model
fig = plt.figure()
model.showEverettFunction(fig)

In [ ]:
# calculate inverse model
invModel = model.invert()

# show everett function of inverse model
fig = plt.figure()
invModel.showEverettFunction(fig)

In [ ]:
def simulate(B):
    output = np.zeros_like(B, dtype=np.float64)
    middle = np.zeros_like(B, dtype=np.float64)
    
    model.setDemagState(80)
    invModel.setDemagState(80)
    
    # Apply input to inverse model and then apply it to non inverse model
    for i in range(len(B)):
        middle[i] = model(B[i])
        output[i] = invModel(middle[i])
    
    fig, axs = plt.subplots(1,3, figsize=(20, 8))
    axs[0].plot(B)
    axs[0].plot(middle)
    axs[0].plot(output)
    axs[0].legend(['input', 'middle', 'output'])

    axs[1].plot(B, middle)
    axs[2].plot(middle, output)
    
    plt.show()
    
    # simulation1 = model.animateHysteresis()
    # simulation2 = invModel.animateHysteresis()

In [ ]:
# Create excitation signal
nSamps = 2500
phi = np.linspace(0, 2 * np.pi + np.pi / 2, nSamps)

sawtooth = np.zeros(nSamps, dtype=np.float64)
sawtooth[phi < np.pi / 2] = 0.7 * 2 / np.pi * phi[phi < np.pi / 2]
sawtooth[np.logical_and(phi < 3 * np.pi / 2, phi > np.pi / 2)] = (
    -0.7 * 2 / np.pi * (phi[np.logical_and(phi < 3 * np.pi / 2, phi > np.pi / 2)] - np.pi)
)
sawtooth[phi > 3 * np.pi / 2] = 0.7 * 2 / np.pi * (phi[phi > 3 * np.pi / 2] - 2 * np.pi)

B = 0.15 * np.sin(30 * phi) + sawtooth

simulate(B)

In [ ]:
from mc2.data_management import FrequencySet, MaterialSet, DataSet

In [ ]:
dataset = DataSet.load_from_file(pathlib.Path("../../data/processed") / "ten_mat_data.pickle")

In [ ]:
B_trajectory = dataset[0][0].B[0, :10_000]
H_trajectory = dataset[0][0].H[0, :10_000]

In [ ]:
H_est = np.zeros_like(B_trajectory, dtype=np.float64)
invModel.setDemagState(80)
    
# Apply input to inverse model and then apply it to non inverse model
for i in tqdm.tqdm(range(len(B_trajectory))):
    H_est[i] = invModel(B_trajectory[i])

In [ ]:
plt.plot(B_trajectory, H_trajectory)

In [ ]:
plt.plot(B_trajectory, H_est)

---